In [1]:
import os
import numpy as np
import pylab as py
import matplotlib.pyplot as plt
from spisea import synthetic, evolution, atmospheres, reddening, ifmr
from spisea.imf import imf, multiplicity
from matplotlib.colors import Normalize
from matplotlib.cm import ScalarMappable
import csv

# Paths for isochrones and output
iso_dir = 'isochrones/'
output_dir = 'output_diagrams/'

# Ensure output directory exists
os.makedirs(output_dir, exist_ok=True)

# Estimation variables
star_index = 0  # Set which star in the CSV to analyze

# Define static isochrone parameters
dist = 4500
evo_model = evolution.Baraffe15()
atm_func = atmospheres.get_merged_atmosphere
red_law = reddening.RedLawCardelli(3.1)
filt_list = ['jwst,F162M', 'jwst,F182M']
filters = ['m_jwst_F162M', 'm_jwst_F182M']
metallicity = 0
level_ages = np.linspace(1, 10, 19) * 1e6  # Define age array
log_age_arr = np.log10(level_ages)

# Load sample magnitudes and errors, skipping the header row
sample_mags = []
sample_errors = []
with open('./s284.csv', mode='r') as file:
    csvFile = csv.reader(file)
    next(csvFile)  # Skip header row
    for lines in csvFile:
        sample_mags.append([float(lines[1]), float(lines[3])])  # F162M, F182M
        sample_errors.append([float(lines[2]), float(lines[4])])  # ERR162M, ERR182M

# Function to generate isochrone grid
def generate_isochrones(AKs):
    return np.array([
        synthetic.IsochronePhot(log_age, AKs, dist, metallicity=metallicity,
                                evo_model=evo_model, atm_func=atm_func,
                                red_law=red_law, filters=filt_list,
                                iso_dir=iso_dir)
        for log_age in log_age_arr
    ])

# Generate isochrone grids
isochrone_AKs_07 = generate_isochrones(0.7)
isochrone_AKs_00 = generate_isochrones(0.0)

# Deredden the observed flux
filter_wavelengths = {
    "m_jwst_F162M": 1.62,
    "m_jwst_F182M": 1.82
}

# Convert filter names to wavelengths
wavelengths = [filter_wavelengths[f] for f in filters]
AKs_ref = 0.7

# Compute dereddened magnitudes and their uncertainties
sample_mags_dereddened = []
sample_errors_dereddened = []

for (m, sigma_m, wavelength) in zip(sample_mags[star_index], sample_errors[star_index], wavelengths):
    A_lambda = red_law.Cardelli89(wavelength, AKs_ref)
    sigma_A = 0.05 * A_lambda / AKs_ref  # Example uncertainty in AKs
    sigma_dereddened = np.sqrt(sigma_m**2 + sigma_A**2)
    
    sample_mags_dereddened.append(m - A_lambda)
    sample_errors_dereddened.append(sigma_dereddened)

# Plot function
def plot_diagram(isochrone_grid, ref_mags, ref_errors, title, filename):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))
    cmap = plt.get_cmap('coolwarm')
    
    # CMD
    for i, instance in enumerate(isochrone_grid):
        color = cmap(i / (len(isochrone_grid) - 1))
        ax1.plot(instance.points[filters[0]] - instance.points[filters[1]],
                 instance.points[filters[1]], color=color)
    ax1.set_xlabel('F162M - F182M')
    ax1.set_ylabel('F182M')
    ax1.invert_yaxis()
    ax1.grid(True)
    
    # MMD
    for i, instance in enumerate(isochrone_grid):
        color = cmap(i / (len(isochrone_grid) - 1))
        ax2.plot(instance.points[filters[0]], instance.points[filters[1]], color=color)
    ax2.set_xlabel('F162M')
    ax2.set_ylabel('F182M')
    ax2.invert_xaxis()
    ax2.invert_yaxis()
    ax2.grid(True)
    
    # Reference star with error bars
    ax1.errorbar(ref_mags[0] - ref_mags[1], ref_mags[1],
                 xerr=np.sqrt(ref_errors[0]**2 + ref_errors[1]**2),
                 yerr=ref_errors[1], fmt='*', markersize=10, color='gold', label="Reference Star")
    ax2.errorbar(ref_mags[0], ref_mags[1], xerr=ref_errors[0],
                 yerr=ref_errors[1], fmt='*', markersize=10, color='gold', label="Reference Star")
    
    ax1.legend()
    ax2.legend()
    fig.suptitle(title)
    plt.savefig(os.path.join(output_dir, filename))
    plt.close()

# Plot both cases
plot_diagram(isochrone_AKs_07, sample_mags[star_index], sample_errors[star_index],
             "Isochrone Grid with AKs = 0.7", "CMD_MMD_AKs_07.png")
plot_diagram(isochrone_AKs_00, sample_mags_dereddened, sample_errors_dereddened,
             "Isochrone Grid with AKs = 0 and Dereddened Flux", "CMD_MMD_AKs_00_Dereddened.png")


/home/wyz5rge/.local/lib/python3.6/site-packages/pysynphot/locations.py:345: UserWarning: Extinction files not found in /scratch/wyz5rge/models/cdbs/extinction
  warnings.warn('Extinction files not found in %s' % (extdir, ))


Isochrone generation took 1.343040 s.
Making photometry for isochrone: log(t) = 6.00  AKs = 0.70  dist = 4500
     Starting at:  2025-03-06 00:43:17.701777   Usually takes ~5 minutes
Starting filter: jwst,F162M   Elapsed time: 0.00 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  2344 K  m_jwst_F162M = 23.11
Starting filter: jwst,F182M   Elapsed time: 0.74 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  2344 K  m_jwst_F182M = 23.10
      Time taken: 1.46 seconds
Isochrone generation took 1.205161 s.
Making photometry for isochrone: log(t) = 6.18  AKs = 0.70  dist = 4500
     Starting at:  2025-03-06 00:43:20.383540   Usually takes ~5 minutes
Starting filter: jwst,F162M   Elapsed time: 0.00 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  2292 K  m_jwst_F162M = 23.47
Starting filter: jwst,F182M   Elapsed time: 0.72 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  2292 K  m_jwst_F182M = 23.49
      Time taken: 1.44 seconds
Isochrone genera

      Time taken: 1.39 seconds
Changing to logg=5.00 for T=  1631 logg=4.04
Isochrone generation took 1.128803 s.
Making photometry for isochrone: log(t) = 6.95  AKs = 0.70  dist = 4500
     Starting at:  2025-03-06 00:43:59.568532   Usually takes ~5 minutes
Starting filter: jwst,F162M   Elapsed time: 0.00 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  1631 K  m_jwst_F162M = 26.11
Starting filter: jwst,F182M   Elapsed time: 0.71 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  1631 K  m_jwst_F182M = 25.75
      Time taken: 1.40 seconds
Changing to logg=5.00 for T=  1602 logg=4.04
Isochrone generation took 1.126426 s.
Making photometry for isochrone: log(t) = 6.98  AKs = 0.70  dist = 4500
     Starting at:  2025-03-06 00:44:02.107363   Usually takes ~5 minutes
Starting filter: jwst,F162M   Elapsed time: 0.00 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  1602 K  m_jwst_F162M = 26.26
Starting filter: jwst,F182M   Elapsed time: 0.74 seconds
Starting

      Time taken: 1.39 seconds
Changing to logg=5.00 for T=  1773 logg=4.01
Isochrone generation took 1.156599 s.
Making photometry for isochrone: log(t) = 6.88  AKs = 0.00  dist = 4500
     Starting at:  2025-03-06 00:44:41.070004   Usually takes ~5 minutes
Starting filter: jwst,F162M   Elapsed time: 0.00 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  1773 K  m_jwst_F162M = 24.29
Starting filter: jwst,F182M   Elapsed time: 0.70 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  1773 K  m_jwst_F182M = 24.39
      Time taken: 1.39 seconds
Changing to logg=5.00 for T=  1711 logg=4.02
Isochrone generation took 1.192268 s.
Making photometry for isochrone: log(t) = 6.90  AKs = 0.00  dist = 4500
     Starting at:  2025-03-06 00:44:43.664567   Usually takes ~5 minutes
Starting filter: jwst,F162M   Elapsed time: 0.00 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  1711 K  m_jwst_F162M = 24.60
Starting filter: jwst,F182M   Elapsed time: 0.70 seconds
Starting